In [ ]:
import numpy as np

In [ ]:
def sliding_window(spikes, T0, T1, window, step=None):
    """
    Computes the firing rate over sliding windows seperated of step

    Parameters
    ----------
    spikes : np.ndarray
        Sorted array of spike times
    T0 : float
        Beginning of spike train to consider, start of bins
    T1 : float
        End of spike train to consider, end of bins
    window : float
        Size of window to compute firing rate
    step : float
        (optional) How far the window moves between bins

    Returns
    -------
    bin_centers : np.ndarray
        Array of midpoints between bin edges
    rates : np.ndarray
        Array of firing rates in Hz for each bin
    """

    # if no step, define step as the window
    if step is None:
        step = window

    # Create beginnig of window for all bins
    bin_starts = np.arange(T0, T1 - window, step)

    # Determine bin centers for plotting
    bin_centers = bin_starts + window / 2

    # Iterate through bins and determine rate in that bin
    rates = []
    for bs in bin_starts:
        bin_spikes = spikes[(spikes < bs + window) & (spikes >= bs)]
        rates.append(len(bin_spikes) / window)
    return bin_centers, np.array(rates)

In [ ]:
def gaussian(x, s):
    """
    Function that takes in values x and standard deviation s to determine Gaussian distribution function

    Parameters
    ----------
    x : np.ndarray
        x-values to evaluate guassian at each point
    s : float
        standard deviation for the gaussian distribution function

    Returns
    -------
    np.ndarray
        gaussian function evaluated with parameter s at values of x
    """
    return (1 / (np.sqrt(2 * np.pi) * s)) * np.exp(-(x**2) / (2 * s**2))


def gauss_kernel(values, spacing, bandwidth=25 / 1000):
    """
    Function performs a simplified convolution with gaussian kernel.

    Parameters
    -----------
    values : np.ndarray
        array of values to evaluate SDF at, should be spikes
    spacing : np.ndarray
        grid to compute SDF at, should be a desired domain
    bandwidth : float
        (optional) bandwidth of gaussian kernel, default is 25/1000

    Returns
    -------
    np.ndarray
        Spike density function with gaussian kernel
    """
    # Cast inputs into numpy arrays
    values = np.asarray(values)  # spikes
    spacing = np.asarray(spacing)  # domain of SDF

    # initialize SDF
    sdf = np.zeros(len(spacing))

    # Perform convolution
    for j in range(len(values)):
        sdf += gaussian((spacing - values[j]), bandwidth)
    return sdf

In [ ]:
def exp_causal_kernel(t, tau):
    """
    Calculates causal expoential kernel at values t

    Parameters
    ----------
    t : np.ndarray
        t-values to evaluate kernel at each point
    tau : float
        Time constant of exponential kernel

    Returns
    -------
    np.ndarray
        Causal exponential kernel
    """

    # Domain for kernel
    t = np.asarray(t)

    # Values for kernel
    exp_k = np.zeros(len(t))

    # Mask to have causal kernel
    mask = t >= 0

    # Apply the kernel to the current domain
    exp_k[mask] = np.exp(-t[mask] / tau) / tau
    return exp_k


def sdf_exp(values, spacing, tau=25 / 1000):
    """
    Function performs a simplified convolution with causal exponential kernel.

    Parameters
    -----------
    values : np.ndarray
        array of values to evaluate SDF at, should be spikes
    spacing : np.ndarray
        grid to compute SDF at, should be a desired domain
    tau : float
        (optional) Time constant of exponential kernel

    Returns
    -------
    np.ndarray
        Spike density function with causal exponential kernel
    """

    # Cast inputs into numpy arrays
    values = np.asarray(values)  # spikes
    spacing = np.asarray(spacing)  # domain of SDF

    # initialize SDF
    sdf = np.zeros(len(spacing))

    # Perform convolution
    for tj in values:
        sdf += exp_causal_kernel(spacing - tj, tau)
    return sdf

In [ ]:
def exp_alpha(t, tau_r, tau_d):
    """
    Function that returns alpha exponential centered at t with tau_d and tau_r time constants

    Parameters
    ----------
    t : np.ndarray
        t-values to evaluate kernel at each point
    tau_r : float
        time constant for rising portion of exponential
    tau_r : float
        time constant for rising portion of exponential

    Returns
    -------
    np.ndarray
        Alpha exponential evaluated at times t
    """

    # Domain for kernel
    t = np.asarray(t)

    # Values for kernel
    exp_k = np.zeros(len(t))

    # Mask to have causal kernel
    mask = t >= 0

    # Apply the kernel to the current domain
    exp_k[mask] = (1 / (tau_d - tau_r)) * (
        np.exp(-t[mask] / tau_d) - np.exp(-t[mask] / tau_r)
    )
    return exp_k


def sdf_exp_alpha(values, spacing, tau_d=25 / 1000, tau_r=10 / 1000):
    """
    Function performs a simplified convolution with alpha exponential kernel.

    Parameters
    -----------
    values : np.ndarray
        array of values to evaluate SDF at, should be spikes
    spacing : np.ndarray
        grid to compute SDF at, should be a desired domain
    tau_d : float
        (optional) Time constant of exponential decay
    tau_r : float
        (optional) Time constant of exponential rise

    Returns
    -------
    np.ndarray
        Spike density function with alpha exponential kernel
    """

    # Cast inputs into numpy arrays
    values = np.asarray(values)  # spikes
    spacing = np.asarray(spacing)  # domain of SDF

    # initialize SDF
    sdf = np.zeros(len(spacing))

    # Perform convolution
    for tj in values:
        sdf += exp_alpha(spacing - tj, tau_d, tau_r)
    return sdf